In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import optuna
from scipy.stats import rankdata

In [ ]:
def add_advanced_features(df):
    """
    Advanced features specifically designed to push AUC > 0.95
    """
    df = df.copy()
    eps = 1e-8
    
    # 1. Polynomial features for max_cluster_phi (most important feature)
    df['max_cluster_phi_squared'] = df['max_cluster_phi']**2
    df['max_cluster_phi_cubed'] = df['max_cluster_phi']**3
    df['max_cluster_phi_sqrt'] = np.sqrt(np.abs(df['max_cluster_phi']))
    df['max_cluster_phi_log'] = np.log(np.abs(df['max_cluster_phi']) + 1)
    
    # 2. Phi-based regions and interactions
    df['phi_region'] = pd.cut(df['max_cluster_phi'], bins=5, labels=False)
    for region in range(5):
        df[f'phi_region_{region}'] = (df['phi_region'] == region).astype(int)
    
    # Conditional features based on phi
    median_pt = df['total_pt'].median()
    df['phi_pt_interaction'] = df['max_cluster_phi'] * (df['total_pt'] > median_pt).astype(int)
    df['phi_high_pt'] = df['max_cluster_phi'] * (df['total_pt'] > df['total_pt'].quantile(0.75)).astype(int)
    
    # 3. Angular moments and higher-order features
    df['phi_variance'] = df['max_cluster_phi']**2 - df['mean_cluster_phi']**2
    df['eta_variance'] = df['max_cluster_eta']**2 - df['mean_cluster_eta']**2
    df['angular_variance_sum'] = df['phi_variance'] + df['eta_variance']
    df['angular_variance_ratio'] = df['phi_variance'] / (df['eta_variance'] + eps)
    
    # 4. Advanced angular features
    df['eta_phi_correlation'] = df['mean_cluster_eta'] * df['mean_cluster_phi']
    df['angular_asymmetry'] = (df['max_cluster_phi'] - df['mean_cluster_phi']) / (df['max_cluster_phi'] + eps)
    df['angular_kurtosis'] = (df['max_cluster_phi'] - df['mean_cluster_phi'])**4 / (df['std_cluster_pt']**4 + eps)
    
    # 5. N-subjettiness approximations
    # Approximate tau21 and tau32 using cluster information
    df['pseudo_tau21'] = df['cv_cluster_pt'] * df['spatial_spread']
    df['pseudo_tau32'] = df['n_clusters'] * df['spatial_asymmetry'] / (df['spatial_spread'] + eps)
    
    # 6. Complex interactions between top features
    # Based on feature importance: max_cluster_phi, spatial_spread, n_clusters
    df['phi_spread_interaction'] = df['max_cluster_phi'] * df['spatial_spread']
    df['phi_ncluster_interaction'] = df['max_cluster_phi'] * df['n_clusters']
    df['spread_ncluster_interaction'] = df['spatial_spread'] * df['n_clusters']
    df['triple_interaction'] = df['max_cluster_phi'] * df['spatial_spread'] * df['n_clusters']
    
    # 7. Ratios with top features
    df['phi_to_spread_ratio'] = df['max_cluster_phi'] / (df['spatial_spread'] + eps)
    df['spread_to_pt_ratio'] = df['spatial_spread'] / (df['total_pt'] + eps)
    df['phi_normalized_by_pt'] = df['max_cluster_phi'] / (df['total_pt'] + eps)
    
    # 8. Threshold-based features with optimized cuts
    phi_thresholds = np.percentile(df['max_cluster_phi'], [20, 40, 60, 80])
    for i, thresh in enumerate(phi_thresholds):
        df[f'phi_above_p{(i+1)*20}'] = (df['max_cluster_phi'] > thresh).astype(int)
    
    spread_thresholds = np.percentile(df['spatial_spread'], [25, 50, 75])
    for i, thresh in enumerate(spread_thresholds):
        df[f'spread_above_p{(i+1)*25}'] = (df['spatial_spread'] > thresh).astype(int)
    
    # 9. Binned interactions
    df['pt_bin'] = pd.qcut(df['total_pt'], q=4, labels=False)
    df['phi_spread_binned'] = df['phi_region'] * 10 + pd.qcut(df['spatial_spread'], q=5, labels=False)
    
    # 10. Distance-based features
    df['euclidean_spatial'] = np.sqrt(df['max_cluster_eta']**2 + df['max_cluster_phi']**2)
    df['manhattan_spatial'] = np.abs(df['max_cluster_eta']) + np.abs(df['max_cluster_phi'])
    df['chebyshev_spatial'] = np.maximum(np.abs(df['max_cluster_eta']), np.abs(df['max_cluster_phi']))
    
    # Clean up
    df = df.drop(columns=['phi_region', 'pt_bin'], errors='ignore')
    df = df.replace([np.inf, -np.inf], 0)
    df = df.fillna(0)
    
    return df

def add_extreme_features(df):
    """
    Add more aggressive features based on the dominance of max_cluster_phi
    """
    df = df.copy()
    eps = 1e-8
    
    # 1. More aggressive phi transformations
    df['max_cluster_phi_pow4'] = df['max_cluster_phi']**4
    df['max_cluster_phi_pow5'] = df['max_cluster_phi']**5
    df['max_cluster_phi_exp'] = np.exp(np.clip(df['max_cluster_phi'], -10, 10))
    df['max_cluster_phi_tanh'] = np.tanh(df['max_cluster_phi'])
    df['max_cluster_phi_sigmoid'] = 1 / (1 + np.exp(-df['max_cluster_phi']))
    
    # 2. Phi-based clustering features
    # Create fine-grained phi bins
    df['phi_fine_bin'] = pd.cut(df['max_cluster_phi'], bins=20, labels=False)
    phi_bin_means = df.groupby('phi_fine_bin')['max_cluster_phi'].transform('mean')
    df['phi_distance_from_bin_mean'] = np.abs(df['max_cluster_phi'] - phi_bin_means)
    
    # 3. Rank-based features
    df['phi_rank'] = rankdata(df['max_cluster_phi']) / len(df)
    df['spatial_spread_rank'] = rankdata(df['spatial_spread']) / len(df)
    df['combined_rank'] = df['phi_rank'] * df['spatial_spread_rank']
    
    # 4. Extreme value indicators
    phi_percentiles = [1, 5, 10, 90, 95, 99]
    for p in phi_percentiles:
        threshold = np.percentile(df['max_cluster_phi'], p)
        if p < 50:
            df[f'phi_below_p{p}'] = (df['max_cluster_phi'] < threshold).astype(int)
        else:
            df[f'phi_above_p{p}'] = (df['max_cluster_phi'] > threshold).astype(int)
    
    # 5. Multi-feature polynomial interactions (focusing on top features)
    # Create polynomial features for top 3: max_cluster_phi, spatial_spread, n_clusters
    poly_features = df[['max_cluster_phi', 'spatial_spread', 'n_clusters']].values
    poly = PolynomialFeatures(degree=3, include_bias=False)
    poly_transformed = poly.fit_transform(poly_features)
    
    # Add polynomial features with meaningful names
    feature_names = poly.get_feature_names_out(['phi', 'spread', 'ncl'])
    for i, name in enumerate(feature_names):
        if i < 3:  # Skip the original features
            continue
        df[f'poly_{name}'] = poly_transformed[:, i]
    
    # 6. Conditional features based on multiple thresholds
    high_phi = df['max_cluster_phi'] > df['max_cluster_phi'].quantile(0.8)
    high_spread = df['spatial_spread'] > df['spatial_spread'].quantile(0.8)
    many_clusters = df['n_clusters'] > df['n_clusters'].quantile(0.7)
    
    df['high_phi_high_spread'] = (high_phi & high_spread).astype(int)
    df['high_phi_low_spread'] = (high_phi & ~high_spread).astype(int)
    df['low_phi_high_spread'] = (~high_phi & high_spread).astype(int)
    df['high_phi_many_clusters'] = (high_phi & many_clusters).astype(int)
    
    # 7. Ratios with all features
    for col in ['total_pt', 'mean_cluster_pt', 'std_cluster_pt', 'mean_cluster_eta']:
        df[f'max_phi_over_{col}'] = df['max_cluster_phi'] / (df[col] + eps)
        df[f'{col}_over_max_phi'] = df[col] / (df['max_cluster_phi'] + eps)
    
    # 8. Statistical aggregations
    df['phi_plus_spread'] = df['max_cluster_phi'] + df['spatial_spread']
    df['phi_minus_spread'] = df['max_cluster_phi'] - df['spatial_spread']
    df['phi_times_spread_squared'] = df['max_cluster_phi'] * df['spatial_spread']**2
    df['phi_over_spread_squared'] = df['max_cluster_phi'] / (df['spatial_spread']**2 + eps)
    
    # 9. Trigonometric features
    df['sin_phi'] = np.sin(df['max_cluster_phi'])
    df['cos_phi'] = np.cos(df['max_cluster_phi'])
    df['sin_2phi'] = np.sin(2 * df['max_cluster_phi'])
    df['cos_2phi'] = np.cos(2 * df['max_cluster_phi'])
    
    # 10. Difference features
    df['phi_eta_diff'] = df['max_cluster_phi'] - df['max_cluster_eta']
    df['phi_mean_phi_diff'] = df['max_cluster_phi'] - df['mean_cluster_phi']
    df['all_angular_sum'] = df['max_cluster_phi'] + df['max_cluster_eta'] + df['mean_cluster_phi'] + df['mean_cluster_eta']
    
    # Clean up
    df = df.drop(columns=['phi_fine_bin'], errors='ignore')
    df = df.replace([np.inf, -np.inf], 0)
    df = df.fillna(0)
    
    return df

def optimize_xgboost_params(X_train, y_train, X_val, y_val, n_trials=50):
    """
    Use Optuna to find optimal XGBoost parameters
    """
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 200, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 8),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'gamma': trial.suggest_float('gamma', 0, 0.5),
            'reg_alpha': trial.suggest_float('reg_alpha', 0, 2),
            'reg_lambda': trial.suggest_float('reg_lambda', 0, 2),
            'random_state': 42
        }
        params['objective'] = 'binary:logistic'
        params['eval_metric'] = 'auc'
        params['tree_method'] = 'hist'
        
        model = xgb.XGBClassifier(**params)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
        
        y_pred = model.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_pred)
        
        return auc
    
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
    
    print(f"Best AUC: {study.best_value:.4f}")
    print("Best params:", study.best_params)
    
    return study.best_params

def train_with_pseudo_labeling(X_train, y_train, X_val, y_val, X_test, confidence_threshold=0.95):
    """
    Use pseudo-labeling on test set to augment training data
    """
    # Train initial model
    model = xgb.XGBClassifier(
        n_estimators=500,
        max_depth=10,
        learning_rate=0.03,
        subsample=0.8,
        colsample_bytree=0.7,
        random_state=42
    )
    model.fit(X_train, y_train)
    
    # Get predictions on test set
    test_proba = model.predict_proba(X_test)[:, 1]
    
    # Select high-confidence predictions
    high_conf_mask = (test_proba > confidence_threshold) | (test_proba < (1 - confidence_threshold))
    
    if high_conf_mask.sum() > 0:
        # Create pseudo labels
        pseudo_labels = (test_proba > 0.5).astype(int)
        
        # Augment training set
        X_augmented = pd.concat([X_train, X_test[high_conf_mask]], axis=0)
        y_augmented = np.concatenate([y_train, pseudo_labels[high_conf_mask]])
        
        # Retrain on augmented data
        model_augmented = xgb.XGBClassifier(
            n_estimators=800,
            max_depth=10,
            learning_rate=0.02,
            subsample=0.8,
            colsample_bytree=0.7,
            random_state=42
        )
        model_augmented.fit(X_augmented, y_augmented)
        
        # Evaluate
        y_pred = model_augmented.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_pred)
        print(f"Pseudo-labeling AUC: {auc:.4f}")
        
        return model_augmented, auc
    
    return model, roc_auc_score(y_val, model.predict_proba(X_val)[:, 1])

# Modified main function
def load_processed_data_extreme():
    """
    Load data with extreme feature engineering
    """
    # Load training data
    X_train = pd.read_csv('data/train/features/cluster_features.csv')
    y_train = np.load('data/train/labels/labels.npy')
    train_ids = np.load('data/train/ids/ids.npy')
    
    # Load validation data
    X_val = pd.read_csv('data/val/features/cluster_features.csv')
    y_val = np.load('data/val/labels/labels.npy')
    val_ids = np.load('data/val/ids/ids.npy')
    
    # Load test data
    X_test = pd.read_csv('data/test/features/cluster_features.csv')
    test_ids = np.load('data/test/ids/ids.npy')
    
    # Add all features
    X_train = add_derived_features(X_train)
    X_train = add_advanced_features(X_train)
    X_train = add_extreme_features(X_train)
    
    X_val = add_derived_features(X_val)
    X_val = add_advanced_features(X_val)
    X_val = add_extreme_features(X_val)
    
    X_test = add_derived_features(X_test)
    X_test = add_advanced_features(X_test)
    X_test = add_extreme_features(X_test)
    
    return X_train, y_train, train_ids, X_val, y_val, val_ids, X_test, test_ids


In [21]:
if __name__ == "__main__":
    # Load data with extreme features
    print("Loading data with extreme feature engineering...")
    X_train, y_train, train_ids, X_val, y_val, val_ids, X_test, test_ids = load_processed_data_extreme()
    
    print(f"Number of features: {X_train.shape[1]}")
    
    # Feature selection focusing on phi-related features
    feature_importance = []
    for col in X_train.columns:
        if 'phi' in col.lower() or 'spread' in col.lower() or 'poly_' in col:
            feature_importance.append(col)
    
    # Also add other original important features
    other_important = ['n_clusters', 'total_pt', 'cluster_pt_ratio', 'pt_concentration', 
                      'spatial_extent', 'angular_asymmetry', 'pseudo_tau21', 'pseudo_tau32']
    
    selected_features = list(set(feature_importance + other_important))
    selected_features = [f for f in selected_features if f in X_train.columns]
    
    print(f"Selected {len(selected_features)} features")
    
    X_train_selected = X_train
    X_val_selected = X_val
    X_test_selected = X_test
    
    # Option 1: Optimize hyperparameters
    print("\nOptimizing XGBoost parameters...")
    best_params = optimize_xgboost_params(X_train_selected, y_train, X_val_selected, y_val, n_trials=100)
    
    # Train with best parameters
    best_model = xgb.XGBClassifier(**best_params)
    best_params['eval_metric'] = 'auc'
    best_model.set_params(eval_metric='auc')
    best_model.fit(X_train_selected, y_train)
    y_pred_best = best_model.predict_proba(X_val_selected)[:, 1]
    auc_best = roc_auc_score(y_val, y_pred_best)
    print(f"\nOptimized model AUC: {auc_best:.4f}")
    from sklearn.calibration import CalibratedClassifierCV
    calibrated = CalibratedClassifierCV(best_model, method='isotonic', cv=3)
    calibrated.fit(X_train_selected, y_train)
    y_pred_calibrated = calibrated.predict_proba(X_val_selected)[:, 1]
    auc_calibrated = roc_auc_score(y_val, y_pred_calibrated)
    print(f"Calibrated model AUC: {auc_calibrated:.4f}")
    
    # Option 2: Try pseudo-labeling if we have test data
    if len(X_test_selected) > 0:
        print("\nTrying pseudo-labeling...")
        pseudo_model, pseudo_auc = train_with_pseudo_labeling(
            X_train_selected, y_train, X_val_selected, y_val, X_test_selected
        )
    
    # Option 3: Train multiple models with different random seeds and average
    print("\nTraining ensemble with different seeds...")
    predictions = []
    for seed in [42, 123, 456, 789, 1011]:
        model = xgb.XGBClassifier(**best_params)
        model.set_params(random_state=seed)
        model.fit(X_train_selected, y_train)
        pred = model.predict_proba(X_val_selected)[:, 1]
        predictions.append(pred)
    
    # Average predictions
    ensemble_pred = np.mean(predictions, axis=0)
    ensemble_auc = roc_auc_score(y_val, ensemble_pred)
    print(f"Ensemble AUC: {ensemble_auc:.4f}")
    weights = []
    for pred in predictions:
        weights.append(roc_auc_score(y_val, pred))
    weights = np.array(weights)
    weights = weights / weights.sum()
    weighted_ensemble_pred = np.average(predictions, axis=0, weights=weights)
    weighted_ensemble_auc = roc_auc_score(y_val, weighted_ensemble_pred)
    print(f"Weighted Ensemble AUC: {weighted_ensemble_auc:.4f}")
    try:
        from catboost import CatBoostClassifier
        cat_model = CatBoostClassifier(iterations=1000, depth=6, learning_rate=0.03, verbose=False)
        cat_model.fit(X_train_selected, y_train, eval_set=(X_val_selected, y_val))
        cat_pred = cat_model.predict_proba(X_val_selected)[:, 1]
        cat_auc = roc_auc_score(y_val, cat_pred)
        print(f"\nCatBoost AUC: {cat_auc:.4f}")
    except:
        print("CatBoost not installed")

[I 2025-07-24 23:44:06,420] A new study created in memory with name: no-name-2097f72e-7126-4de7-b49f-8855771d5443


Loading data with extreme feature engineering...
Number of features: 136
Selected 91 features

Optimizing XGBoost parameters...


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-07-24 23:44:07,460] Trial 0 finished with value: 0.9379497841036303 and parameters: {'n_estimators': 724, 'max_depth': 4, 'learning_rate': 0.048123991842267365, 'subsample': 0.7427818854367243, 'colsample_bytree': 0.9093127995003304, 'min_child_weight': 3, 'gamma': 0.14873253394028513, 'reg_alpha': 0.05339742713104734, 'reg_lambda': 1.0915799588540303}. Best is trial 0 with value: 0.9379497841036303.
[I 2025-07-24 23:44:09,231] Trial 1 finished with value: 0.9387494002878619 and parameters: {'n_estimators': 740, 'max_depth': 8, 'learning_rate': 0.029191275110726736, 'subsample': 0.5261520459694444, 'colsample_bytree': 0.9258138443467556, 'min_child_weight': 3, 'gamma': 0.27681444727262444, 'reg_alpha': 1.704658034791571, 'reg_lambda': 0.17937377169331592}. Best is trial 1 with value: 0.9387494002878619.
[I 2025-07-24 23:44:10,628] Trial 2 finished with value: 0.9399754784370169 and parameters: {'n_estimators': 842, 'max_depth': 7, 'learning_rate': 0.036122257483936965, 'subsamp

In [18]:
# Use the best_model which gave you 0.9470 AUC
test_predictions = best_model.predict_proba(X_test_selected)[:, 1]
solution = pd.DataFrame({'id': test_ids, 'label': test_predictions})
solution.to_csv('solution.csv', index=False)

In [19]:
# Generate predictions from all models in the ensemble
ensemble_test_predictions = []
for seed in [42, 123, 456, 789, 1011]:
    model = xgb.XGBClassifier(**best_params)
    model.set_params(random_state=seed)
    model.fit(X_train_selected, y_train)
    pred = model.predict_proba(X_test_selected)[:, 1]
    ensemble_test_predictions.append(pred)

# Use the same weights from validation
weights = []
for seed in [42, 123, 456, 789, 1011]:
    model = xgb.XGBClassifier(**best_params)
    model.set_params(random_state=seed)
    model.fit(X_train_selected, y_train)
    val_pred = model.predict_proba(X_val_selected)[:, 1]
    weights.append(roc_auc_score(y_val, val_pred))
weights = np.array(weights)
weights = weights / weights.sum()

# Weighted average
test_predictions = np.average(ensemble_test_predictions, axis=0, weights=weights)
solution = pd.DataFrame({'id': test_ids, 'label': test_predictions})
solution.to_csv('solution_ensemble.csv', index=False)